In [7]:
import shutil
import random
import yaml
from pathlib import Path

# ─── CONFIG ──────────────────────────────────────────────────────────────────────
NVD_ROOT            = Path('../nvd/nvd_dataset_full')  # must have images/{train,val,test}, labels/{train,val,test}
PCNVD_LABEL_DIR     = Path('PCNVD/Labels')
PCNVD_IMAGE_DIR     = Path('PCNVD/Images_BIGBOY')
OUT_ROOT            = Path('combined_yolo_dataset_full_no_aug')
CLASS_NAMES         = ['car']
MAX_BACKGROUND_RATIO = 0.01  # Maximum 20% of train images can be background-only

# ─── HELPERS ──────────────────────────────────────────────────────────────────────

def copy_split_from_nvd():
    """Copy pre-split NVD dataset to output directory, and count background-only train images."""
    nvd_background_count = 0
    nvd_train_count = 0

    for split in ['train', 'val', 'test']:
        img_dir = NVD_ROOT / 'images' / split
        lbl_dir = NVD_ROOT / 'labels' / split

        for sub in [f'images/{split}', f'labels/{split}']:
            (OUT_ROOT / sub).mkdir(parents=True, exist_ok=True)

        img_files = sorted(img_dir.glob('*.[jp][pn]g'))
        for img_path in img_files:
            stem = img_path.stem
            lbl_path = lbl_dir / f"{stem}.txt"
            dst_img = OUT_ROOT / f'images/{split}/{img_path.name}'
            dst_lbl = OUT_ROOT / f'labels/{split}/{stem}.txt'

            shutil.copy(img_path, dst_img)
            if lbl_path.exists():
                shutil.copy(lbl_path, dst_lbl)
                is_empty = lbl_path.stat().st_size == 0
            else:
                dst_lbl.write_text("")
                is_empty = True

            if split == 'train':
                nvd_train_count += 1
                if is_empty:
                    nvd_background_count += 1

    return nvd_train_count, nvd_background_count

def augment_train_with_pcnvd(current_total, current_background):
    """Augment training set with PCNVD images and labels, limiting total background ratio."""
    pcnvd_imgs = sorted([p for p in PCNVD_IMAGE_DIR.iterdir() if p.suffix.lower() in ['.jpg', '.png']])
    random.shuffle(pcnvd_imgs)

    total = current_total
    background = current_background
    added = 0

    for img_path in pcnvd_imgs:
        stem = img_path.stem
        lbl_path = PCNVD_LABEL_DIR / f"{stem}.txt"
        dst_img = OUT_ROOT / f"images/train/{img_path.name}"
        dst_lbl = OUT_ROOT / f"labels/train/{stem}.txt"

        if dst_img.exists():
            # Allow duplicates by suffix
            i = 1
            while dst_img.exists():
                dst_img = OUT_ROOT / f"images/train/{stem}_{i}{img_path.suffix}"
                dst_lbl = OUT_ROOT / f"labels/train/{stem}_{i}.txt"
                i += 1

        if lbl_path.exists() and lbl_path.stat().st_size > 0:
            shutil.copy(img_path, dst_img)
            shutil.copy(lbl_path, dst_lbl)
            total += 1
            added += 1
        else:
            # Check if adding this would exceed 20% background limit
            if (background + 1) / (total + 1) <= MAX_BACKGROUND_RATIO:
                shutil.copy(img_path, dst_img)
                dst_lbl.write_text("")
                background += 1
                total += 1
                added += 1
            # else: skip this background image

    print(f"➕ Augmented {added} PCNVD images into training set.")
    print(f"📊 Final train set: {total} images, {background} background-only ({(background / total):.2%})")

def write_data_yaml():
    data = {
        'path': str(OUT_ROOT.resolve()),
        'train': 'images/train',
        'val':   'images/val',
        'test':  'images/test',
        'names': CLASS_NAMES
    }
    with open(OUT_ROOT / 'data.yaml', 'w') as f:
        yaml.dump(data, f, sort_keys=False)

def prepare_dataset():
    if OUT_ROOT.exists():
        shutil.rmtree(OUT_ROOT)
    print("📦 Copying main dataset from NVD...")
    nvd_total, nvd_background = copy_split_from_nvd()
    print(f"📊 NVD train images: {nvd_total}, background-only: {nvd_background} ({(nvd_background / nvd_total):.2%})")
    # print("🔁 Augmenting training set with PCNVD (enforcing 1% max background)...")
    # augment_train_with_pcnvd(nvd_total, nvd_background)
    write_data_yaml()
    print(f"✅ Dataset prepared under {OUT_ROOT}")


if __name__ == "__main__":
    random.seed(42)
    prepare_dataset()


📦 Copying main dataset from NVD...
📊 NVD train images: 4355, background-only: 0 (0.00%)
✅ Dataset prepared under combined_yolo_dataset_full_no_aug


In [9]:
import random
import sys
from pathlib import Path
from ultralytics import YOLO
import yaml

# Configuration constants
OUT_ROOT    = Path('combined_yolo_dataset_full_no_aug')
EXPERIMENT  = 'yolov8m_big_NVD_augment_less_background'
EPOCHS      = 200
IMGSZ       = 640
BATCH       = 16
FREEZE_BACKBONE = False
PATIENCE    = 10
CONF_THRESH = 0.001  # or 0.01 is okay
IOU_THRESH = 0.50



def train():
    """Train YOLOv8 with custom hyperparameters."""
    model = YOLO('yolov8m.pt')

    # === Commented out fine-tuning (hyperparameter search) ===
    # model.tune(
    #     data=str(OUT_ROOT / 'data.yaml'),
    #     epochs=EPOCHS,
    #     imgsz=IMGSZ,
    #     batch=BATCH,
    #     name=EXPERIMENT + '_tune',
    #     freeze=FREEZE_BACKBONE,
    #     optimizer='AdamW',
    #     augment=True,
    #     patience=PATIENCE,
    #     iterations=HYP_TUNE_TRIALS
    # )

    # === Manual hyperparameters for training ===
    hyp = {
        'lr0': 0.003,
        'lrf': 0.1,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3.0,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 0.5,
        'cls': 0.05,
        'dfl': 1.5,
        'label_smoothing': 0.0,
        'nbs': 64,
    
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 5.0,
        'translate': 0.1,
        'scale': 0.4,
        'shear': 2.0,
        'perspective': 0.001,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.2,
        'copy_paste': 0.1
    }

    
    hyp_path = OUT_ROOT / 'hyp.yaml'
    with open(hyp_path, 'w') as f:
        yaml.dump(hyp, f)

    model.train(
        data=str(OUT_ROOT / 'data.yaml'),
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        name=EXPERIMENT,
        freeze=FREEZE_BACKBONE,
        optimizer='AdamW',
        augment=True,
        patience=PATIENCE,
        cfg=str(hyp_path)  # Load hyp from YAML
    )


def infer():
    """Run inference using the trained weights."""
    weights = Path('runs') / 'train' / EXPERIMENT / 'weights' / 'best.pt'
    if not weights.exists():
        sys.exit(f"ERROR: checkpoint not found at {weights}")

    model = YOLO(str(weights))
    results = model.predict(
        source=str(OUT_ROOT / 'images/val'),
        imgsz=IMGSZ,
        conf=CONF_THRESH,
        iou=IOU_THRESH,
        save=True
    )

    print("\nInference Summary:")
    for r in results:
        confs = r.boxes.conf.cpu().numpy() if r.boxes.conf.nelement() else []
        print(f"{r.orig_shape} → {len(confs)} detections, scores: {confs}")


if __name__ == "__main__":
    random.seed(42)
    train()
    infer()


New https://pypi.org/project/ultralytics/8.3.140 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.137 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11005MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=0.5, cache=False, cfg=combined_yolo_dataset_full_no_aug/hyp.yaml, classes=None, close_mosaic=10, cls=0.05, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined_yolo_dataset_full_no_aug/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=False, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.1, mask_ratio=4, max

train: Scanning /Project_NVD/combined_yolo_dataset_full_no_aug/labels/train... 4355 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4355/4355 [07:11<00:00, 10.08it/s]


train: New cache created: /Project_NVD/combined_yolo_dataset_full_no_aug/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 16.0±7.8 MB/s, size: 3555.1 KB)


val: Scanning /Project_NVD/combined_yolo_dataset_full_no_aug/labels/val... 2904 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2904/2904 [05:19<00:00,  9.09it/s]


val: New cache created: /Project_NVD/combined_yolo_dataset_full_no_aug/labels/val.cache
Plotting labels to runs/detect/yolov8m_big_NVD_augment_less_background2/labels.jpg... 
optimizer: AdamW(lr=0.003, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/yolov8m_big_NVD_augment_less_background2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


74 epochs completed in 1.921 hours.
Optimizer stripped from runs/detect/yolov8m_big_NVD_augment_less_background2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/yolov8m_big_NVD_augment_less_background2/weights/best.pt, 52.0MB

Validating runs/detect/yolov8m_big_NVD_augment_less_background2/weights/best.pt...
Ultralytics 8.3.137 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11005MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 91/91 [00:28<00:00,  3.14it/s]


                   all       2904       9175      0.938      0.948      0.981      0.713
Speed: 0.1ms preprocess, 6.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/yolov8m_big_NVD_augment_less_background2


SystemExit: ERROR: checkpoint not found at runs/train/yolov8m_big_NVD_augment_less_background/weights/best.pt

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from IPython.display import Image, display, Markdown
import cv2
import yaml

# ─── CONFIG ─────────────────────────────────────────────────────────────────────
WEIGHTS       = "runs/detect/yolov8m_big_NVD_augment_less_background_best/weights/best.pt"
DATA_YAML     = "combined_yolo_dataset_full/data.yaml"
IMG_ROOT      = Path("../nvd/nvd_dataset_full/images/test")
LABEL_DIR     = Path("../nvd/nvd_dataset_full/labels/test")
OUTPUT_DIR    = Path("annotated_tests")
CONF_THRESH   = 0.5
IOU_THRESH = 0.5  # threshold to consider a prediction as a correct match

CLASS_ID      = 0  # 'car'
NUM_VIS       = 30  # show top 5 images with most GT
# ────────────────────────────────────────────────────────────────────────────────






# ─── LOAD MODEL ─────────────────────────────────────────────────────────────────
model = YOLO(WEIGHTS)

# ─── RUN EVALUATION ─────────────────────────────────────────────────────────────
results = model.val(data=DATA_YAML, conf=CONF_THRESH, split='test')


# Resultatobjekt innehåller metrik i results.box (av typen Metric)
metrics = results.box

# Hämta precision, recall, AP@0.5, AP@0.5:0.95 för en specifik klass
precision, recall, ap50, ap = metrics.class_result(CLASS_ID)

# Skriv ut
print("\n🔍 Evaluation Metrics:")
print(f"📈 Precision       : {precision:.4f}")
print(f"📉 Recall          : {recall:.4f}")
print(f"🏁 AP@0.5          : {ap50:.4f}")
print(f"📊 AP@0.5:0.95     : {ap:.4f}")
print(f"🎯 mAP@0.5 (mean)  : {metrics.map50:.4f}")
print(f"🎯 mAP@0.5:0.95    : {metrics.map:.4f}")


random.seed(15)  # For reproducibility


def get_gt_boxes(label_file):
    boxes = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) == CLASS_ID:
                _, x_center, y_center, width, height = map(float, parts)
                boxes.append((x_center, y_center, width, height))
    return boxes

def yolo_to_xyxy(box, img_w, img_h):
    x_c, y_c, w, h = box
    x1 = int((x_c - w / 2) * img_w)
    y1 = int((y_c - h / 2) * img_h)
    x2 = int((x_c + w / 2) * img_w)
    y2 = int((y_c + h / 2) * img_h)
    return [x1, y1, x2, y2]

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# Step 1: Find valid images with GT of CLASS_ID
valid_images = []
for img_path in sorted(IMG_ROOT.glob("*.*")):
    if img_path.suffix.lower() not in ('.jpg', '.jpeg', '.png'):
        continue
    lbl_path = LABEL_DIR / f"{img_path.stem}.txt"
    if not lbl_path.exists():
        continue
    if get_gt_boxes(lbl_path):
        valid_images.append(img_path)

# Step 2: Deterministically select 10 images
selected_images = random.sample(valid_images, NUM_VIS)

# Step 3: Annotate each selected image
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

for idx, img_path in enumerate(selected_images, 1):
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # Ground Truth boxes
    gt_boxes_yolo = get_gt_boxes(LABEL_DIR / f"{img_path.stem}.txt")
    gt_boxes = [yolo_to_xyxy(box, w, h) for box in gt_boxes_yolo]

    # Predictions
    results = model.predict(str(img_path), conf=CONF_THRESH)[0]

    # Draw predictions (green)
    for box, cls_id, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
        if int(cls_id) != CLASS_ID:
            continue
        x1, y1, x2, y2 = map(int, box.cpu().numpy())
        label = f"{model.names[CLASS_ID]} {conf:.2f}"
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        ts = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        cv2.rectangle(img_rgb, (x1, y1 - ts[1] - 4), (x1 + ts[0], y1), (0, 255, 0), -1)
        cv2.putText(img_rgb, label, (x1, y1 - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Draw ground truth boxes (blue)
    for gb in gt_boxes:
        x1, y1, x2, y2 = gb
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img_rgb, "GT", (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    out_path = OUTPUT_DIR / f"sample_{idx:02d}_{img_path.name}"
    cv2.imwrite(str(out_path), cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

    display(Markdown(f"### {idx}. {img_path.name}"))
    display(Image(filename=str(out_path)))